In [1]:
import os
import numpy as np
import pandas as pd
from utils.helper_functions import calculate_cbtmin, calculate_sleep_onset

In [2]:
save_path = f"data/chang14_selected_sleep_parameters"
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [3]:
selected_parameters = pd.read_csv(f"data/sleep_parameter_exploration/selected_sleep_parameters.csv",
                                  index_col=0)

In [4]:
time = np.load("data/light_schedules/time.npy")

In [39]:
all_reading_conditions = [
    'chang14_ebook_first', 'chang14_ebook_second',
    'realistic_ebook_first', 'realistic_ebook_second',
]
simulation_data_path = "data/chang14_selected_sleep_parameters"
simulation_features = pd.DataFrame()
for idx in selected_parameters.index:
    data = np.load(f"{simulation_data_path}/sleep_parameters_chang14_simulation_{idx}.npz", 
                    allow_pickle=True)

    regular_sleep_onset = calculate_sleep_onset(time, data, 'regular')
    regular_cbtmin = calculate_cbtmin(time, data, 'regular')

    # Check if they have the same size
    if len(regular_sleep_onset) != len(regular_cbtmin):
        # fix the size
        if len(regular_sleep_onset) > len(regular_cbtmin):
            regular_sleep_onset = regular_sleep_onset[:len(regular_cbtmin)]
        else:
            regular_cbtmin = regular_cbtmin[:len(regular_sleep_onset)]

    normalized_sleep_onset = regular_sleep_onset - np.arange(1, len(regular_sleep_onset) + 1) * 24 + 24
    normalized_cbtmin = regular_cbtmin - np.arange(1, len(regular_cbtmin) + 1) * 24 + 24
    normalized_sleep_latency = (normalized_sleep_onset - 22.0) * 60
    normalized_sleep_latency = (sleep_onset % 24 - 22.0) * 60
    normalized_sleep_latency[normalized_sleep_latency < 0] = 0.0

    data_subset = {
        'simulation_idx': idx,
        'day': np.arange(1, len(regular_cbtmin) + 1),
        'simulation_condition': 'regular',
        'sleep_onset': normalized_sleep_onset,
        'cbtmin': normalized_cbtmin,
        'sleep_latency_minutes': normalized_sleep_latency,
    }

    simulation_features = pd.concat([simulation_features, pd.DataFrame(data_subset)])

    for reading_condition in all_reading_conditions:
        sleep_onset = calculate_sleep_onset(time, data, reading_condition)
        cbtmin = calculate_cbtmin(time, data, reading_condition)

        if len(sleep_onset) > len(cbtmin):
            sleep_onset = sleep_onset[:len(cbtmin)]
        else:
            cbtmin = cbtmin[:len(sleep_onset)]

        normalized_sleep_onset = sleep_onset - np.arange(1, len(sleep_onset) + 1) * 24 + 24
        normalized_cbtmin = cbtmin - np.arange(1, len(cbtmin) + 1) * 24 + 24
        normalized_sleep_latency = (normalized_sleep_onset - 22.0) * 60
        normalized_sleep_latency = (sleep_onset % 24 - 22.0) * 60
        normalized_sleep_latency[normalized_sleep_latency < 0] = 0.0

        data_subset = {
            'simulation_idx': idx,
            'day': np.arange(1, len(cbtmin) + 1),
            'simulation_condition': reading_condition,
            'sleep_onset': normalized_sleep_onset,
            'cbtmin': normalized_cbtmin,
            'sleep_latency_minutes': normalized_sleep_latency,
        }
        simulation_features = pd.concat([simulation_features, pd.DataFrame(data_subset)])

In [40]:
simulation_features.to_csv(f"{save_path}/selected_sleep_parameters_simulation_features.csv",
                           index=False)